# ステップ1-3: データ準備（HTML取得→パース→特徴量追加→fixed.csv保存）

このノートブックでは以下の処理を行います：
1. 最新レースHTMLの一括ダウンロード
2. データ確認
3. **HTMLパース→特徴量計算→features_dataset_fixed.csv保存（統合処理）**

**使い方**: 上から順番にセルを実行してください（`Shift + Enter`）

**差分更新**: ステップ3では、既存のfeatures_dataset_fixed.csvがある場合、新規HTMLのみ処理します（高速）

---
## ステップ1: 環境設定

In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# pythonディレクトリをパスに追加
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
python_dir = os.path.join(notebook_dir, '..', 'python')
sys.path.insert(0, os.path.abspath(python_dir))

# 必要なモジュールのインポート
import pandas as pd
from datetime import datetime
import time
import requests
from pathlib import Path

print("✅ 環境設定完了")
print(f"Python path: {os.path.abspath(python_dir)}")

✅ 環境設定完了
Python path: /Users/iinumac/Develop/Keiba/python


---
## ステップ2: 最新レースHTMLの一括ダウンロード

netkeibaから最新のレースHTMLを自動的にダウンロードします。
- 既に存在するHTMLはスキップ
- レースが存在しない場合は次へスキップ
- 当年のみ

In [2]:
def generate_race_ids(start_year=2010, end_year=None, debug=False):
    """
    レースIDを生成するジェネレータ
    
    race_id形式: YYYYCCRRDDNN (12桁)
        YYYY: 年 (2010-現在)
        CC: 競馬場コード (01-10)
        RR: 開催回 (01-10)
        DD: 開催日 (01-20)
        NN: レース番号 (01-12)
    """
    if end_year is None:
        end_year = datetime.now().year
    
    for year in range(start_year, end_year + 1):
        for venue_code in range(1, 11):  # 01-10
            kaisai = 1
            while kaisai <= 10:  # 01-10（最大10回）
                day = 1
                kaisai_has_races = False
                skip_kaisai_flag = False
                
                while day <= 20:  # 01-20（最大20日）
                    race_num = 1
                    
                    while race_num <= 12:  # 01-12
                        race_id = f"{year:04d}{venue_code:02d}{kaisai:02d}{day:02d}{race_num:02d}"
                        skip_signal = yield race_id
                        
                        if skip_signal == 'race_found':
                            kaisai_has_races = True
                        elif skip_signal == 'skip_day':
                            break
                        elif skip_signal == 'skip_kaisai':
                            skip_kaisai_flag = True
                            break
                        
                        race_num += 1
                    
                    if skip_kaisai_flag:
                        break
                    day += 1
                
                if skip_kaisai_flag:
                    kaisai += 1
                    continue
                
                if not kaisai_has_races:
                    break
                
                kaisai += 1

def download_race_html(race_id, output_dir='../data/raceHTML', force=False):
    """
    レースHTMLをダウンロード
    """
    year = race_id[:4]
    output_path = Path(output_dir) / year / f"{race_id}.html"
    
    if output_path.exists() and not force:
        return 'exists'
    
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    url = f"https://db.netkeiba.com/race/{race_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            return 'not_found'
        
        response.encoding = 'EUC-JP'
        html_text = response.text
        
        if len(html_text) < 5000:
            return 'not_found'
        
        if 'データが存在しません' in html_text or 'お探しのページが見つかりませんでした' in html_text:
            return 'not_found'
        
        horse_link_count = html_text.count('/horse/')
        if horse_link_count < 3:
            return 'not_found'
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html_text)
        
        return 'success'
        
    except requests.exceptions.Timeout:
        return 'error'
    except Exception as e:
        return 'error'

print("✅ ダウンロード関数準備完了")

✅ ダウンロード関数準備完了


In [3]:
# HTMLダウンロードの実行
print("📥 レースHTMLのダウンロードを開始します")
print("⚠️ この処理には時間がかかる場合があります\n")

# ダウンロード開始年の決定（当年のみ）
current_year = datetime.now().year
    
start_year = current_year
print(f"当年のみ更新: {start_year}年～現在\n")

# ダウンロード実行
generator = generate_race_ids(start_year=start_year)
stats = {'success': 0, 'exists': 0, 'not_found': 0, 'error': 0}
last_success_race = None
total_processed = 0

try:
    race_id = next(generator)
    
    while True:
        result = download_race_html(race_id)
        stats[result] += 1
        total_processed += 1
        
        skip_signal = None
        
        if result == 'success' or result == 'exists':
            last_success_race = race_id
            skip_signal = 'race_found'
            if result == 'success':
                print(f"✅ {race_id}: ダウンロード成功")
                time.sleep(0.5)
        elif result == 'not_found':
            race_num = int(race_id[-2:])
            if race_num == 1:
                day = int(race_id[-4:-2])
                if day == 1:
                    skip_signal = 'skip_kaisai'
                else:
                    skip_signal = 'skip_day'
        
        if total_processed % 100 == 0:
            print(f"\n📊 進捗: {total_processed}件処理")
            print(f"   成功: {stats['success']}, 既存: {stats['exists']}")
            print(f"   未発見: {stats['not_found']}, エラー: {stats['error']}\n")
        
        race_id = generator.send(skip_signal)
        
except StopIteration:
    print("\n✅ ダウンロード完了（全race_id処理済み）")

print(f"\n📊 最終統計:")
print(f"   処理件数: {total_processed}件")
print(f"   新規ダウンロード: {stats['success']}件")
print(f"   既存スキップ: {stats['exists']}件")
print(f"   未発見: {stats['not_found']}件")
print(f"   エラー: {stats['error']}件")
if last_success_race:
    print(f"   最新レース: {last_success_race}")

📥 レースHTMLのダウンロードを開始します
⚠️ この処理には時間がかかる場合があります

当年のみ更新: 2025年～現在


📊 進捗: 100件処理
   成功: 0, 既存: 100
   未発見: 0, エラー: 0


📊 進捗: 200件処理
   成功: 0, 既存: 191
   未発見: 9, エラー: 0


📊 進捗: 300件処理
   成功: 0, 既存: 269
   未発見: 31, エラー: 0


📊 進捗: 400件処理
   成功: 0, 既存: 369
   未発見: 31, エラー: 0


📊 進捗: 500件処理
   成功: 0, 既存: 452
   未発見: 48, エラー: 0


📊 進捗: 600件処理
   成功: 0, 既存: 538
   未発見: 62, エラー: 0


📊 進捗: 700件処理
   成功: 0, 既存: 621
   未発見: 79, エラー: 0


📊 進捗: 800件処理
   成功: 0, 既存: 705
   未発見: 95, エラー: 0


📊 進捗: 900件処理
   成功: 0, 既存: 793
   未発見: 107, エラー: 0


📊 進捗: 1000件処理
   成功: 0, 既存: 865
   未発見: 135, エラー: 0


📊 進捗: 1100件処理
   成功: 0, 既存: 943
   未発見: 157, エラー: 0


📊 進捗: 1200件処理
   成功: 0, 既存: 1031
   未発見: 169, エラー: 0


📊 進捗: 1300件処理
   成功: 0, 既存: 1131
   未発見: 169, エラー: 0


📊 進捗: 1400件処理
   成功: 0, 既存: 1220
   未発見: 180, エラー: 0


📊 進捗: 1500件処理
   成功: 0, 既存: 1308
   未発見: 192, エラー: 0


📊 進捗: 1600件処理
   成功: 0, 既存: 1399
   未発見: 201, エラー: 0


📊 進捗: 1700件処理
   成功: 0, 既存: 1482
   未発見: 218, エラー: 0


📊 進捗: 1800件処理
   成功: 0, 既存: 1